# Group 36 Project Proposal

### Group Members: Bhavtej Bhasin, Peter Chen, Theresa Choi, Sky Langille
### DSCI 100 004

**Proposed Title:**

**Introduction:**
- provide background info on selected topic
- clearly state our predictive question
- identify and describe the data set we chose to use

**Preliminary Data Analysis:**
- demonstrate data set can be read into R
- clean and wrangle data into tidy format
- separate into training and test data; summarize training data into a table and visualize data with at least one plot

**Methods:**
- explain how we will conduct analysis and what variable or columns will be used
- describe at least one way we will visualize our results

**Expected Outcomes and Significance:**
- what do we expect to happen?
- why is this important?
- what future questions can this lead to?